In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import models
from torch.optim import lr_scheduler

import glob
from skimage import io, transform
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import time
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 3

print('using device:', device)

In [ ]:
class GripForceDataset(Dataset):
    """Finger Grip Force dataset."""
    def __init__(self, root_dir, n_subj, n_exp_per_subj, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            n_subj (int): Number of test subjects.
            n_exp_per_subj (int): Number of experiments per test subject.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.N = 0
        self.images = np.zeros((0, 3), dtype=np.int)
        self.force = np.zeros((0, 2), dtype=np.float)
        for subject_number in n_subj:
            for experiment_number in n_exp_per_subj:
                img_path = '{}/{:02d}/{:02d}/frames_aligned/'.format(root_dir, subject_number, experiment_number)
                force_path = '{}/{:02d}/{:02d}/newtons.csv'.format(root_dir, subject_number, experiment_number)
        
                # load image paths from current experiment
                n = len(glob.glob('{}*.png'.format(img_path)))
                experiment = np.zeros((n, 3), dtype=np.int)
                experiment[:,0] = subject_number
                experiment[:,1] = experiment_number
                experiment[:,2] = np.arange(n)
                
                # load force data from current experiment
                force = np.loadtxt(force_path, delimiter=',')
                
                # store image paths and force data (2 labels per image)
                self.images = np.concatenate((self.images, experiment), axis=0)
                self.force = np.concatenate((self.force, force), axis=0)
                self.N += n
                    
    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        img_path = '{}/{:02d}/{:02d}/frames_small/{:04d}.png'.format(self.root_dir, 
                                                                    self.images[idx, 0], 
                                                                    self.images[idx, 1], 
                                                                    self.images[idx, 2])
        frames = io.imread(img_path)
        frames = np.array(frames, dtype=np.float) / 255.0
        
        force = self.force[idx] / 30.0 - 0.5
        
        sample = {'frames': frames, 'force': force}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [ ]:
# test Dataset class
root_dir = '/media/viktor/Samsung_T5/Research/dataset'
n_subj = [4]
n_exp_per_subj = [3,1,2,4,5,6,7,8,9,10]

ds = GripForceDataset(root_dir, n_subj, n_exp_per_subj)

print('len={}'.format(ds.__len__()))

fig = plt.figure(figsize=(20, 20))
for i in range(5):
    sample = ds[0 + i]
    ax = plt.subplot(1, 6, i + 1)
    ax.set_title('frame#{}'.format(i))
    ax.axis('off')
    plt.imshow(sample['frames'])
    print(sample['frames'].shape)
plt.show()

In [ ]:
# sample = ds[500]

# import scipy.misc
# scipy.misc.imsave('sample3.jpg', sample['frames'])

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        frames, force = sample['frames'], sample['force']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        frames = frames.transpose((2, 0, 1))
        return {'frames': torch.from_numpy(frames).float(),
                'force': torch.from_numpy(force).float()}

class Normalize(object):
    mean=[0.0, 0.0, 0.0]
    std=[1.0, 1.0, 1.0]
    
    # subj 1
#     mean=[0.5695, 0.4747, 0.4095]
#     std=[0.1545, 0.1762, 0.1931]

    # subj 2
#     mean = [0.5308, 0.4349, 0.3549]
#     std = [0.1436, 0.1596, 0.1730]

    # subj 3
#     mean = [0.5212, 0.4471, 0.4404]
#     std = [0.1569, 0.1885, 0.2102]

    # subj 4
#     mean = [0.5207, 0.4191, 0.4206]
#     std = [0.1518, 0.1816, 0.1995]

    # subj 5
    mean = [0.5207, 0.4191, 0.4206]
    std = [0.1518, 0.1816, 0.1995]
    
    def __call__(self, sample):
        frames, force = sample['frames'], sample['force']

        tr = T.Normalize(self.mean, self.std)
        
        frames = tr(frames)
        return {'frames': frames, 'force': force}

In [ ]:
normalize = True

# test Dataset with transform
if normalize == True:
    transform = T.Compose([ToTensor(), Normalize()])
else:
    transform = ToTensor()
    
transformed_dataset = GripForceDataset(root_dir, n_subj, n_exp_per_subj, transform=transform)

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['frames'].size(), sample['force'].size())

    if i == 3:
        break

In [ ]:
### Split data into training, validation, and testing sets

batch_size = 128
num_workers = 64
val_split = .05
test_split = .1
shuffle_dataset = False # Only for train/val - test dataset is not shuffled
random_seed = 42

# Data indices
dataset_size = len(transformed_dataset)

test_size = int(np.floor(test_split * dataset_size))
val_size = int(np.floor(val_split * dataset_size))
train_size = dataset_size - val_size - test_size

indices = list(range(test_size, dataset_size))

if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices = indices[:train_size]
val_indices = indices[train_size:]
if not shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(train_indices)

# # JUST FOR DEBUG PURPOSE
# batch_size = 3
# train_indices = train_indices[:6]
# val_indices = val_indices[:6]
# test_indices = test_indices[:6]

# Data Samplers
train_sampler = sampler.SubsetRandomSampler(train_indices)
val_sampler = sampler.SubsetRandomSampler(val_indices)
test_sampler = sampler.SequentialSampler(range(test_size))

# Data Loaders
train_loader = torch.utils.data.DataLoader(transformed_dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(transformed_dataset,
                                         batch_size=1,
                                         num_workers=num_workers,
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(transformed_dataset,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          sampler=test_sampler)

dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [ ]:
## Calculate mean and standard deviation for Normalization

mean = 0.
std = 0.
nb_samples = 0.
for data in dataloaders['train']:
    x = data['frames']
    batch_samples = x.size(0)
    x = x.view(batch_samples, x.size(1), -1)
    mean += x.mean(2).sum(0)
    std += x.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
print(mean, std)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    global loss_stat

    best_model_wts = copy.deepcopy(model.state_dict())
    best_mse = 1e10

    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            phase_since = time.time()
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for sample in dataloaders[phase]:
                inputs, labels = sample['frames'], sample['force']
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        loss_stat['train_'].append([epoch, loss.item()])

                # statistics
                running_loss += loss.item() * inputs.size(0)

#             epoch_loss = running_loss / dataset_sizes[phase]
            epoch_loss = running_loss / len(dataloaders[phase].sampler.indices)
            epoch_mse = epoch_loss * 30.0

            print('{} Loss: {:.4f} MSE: {:.4f}'.format(phase, epoch_loss, epoch_mse))
            now = time.time()
            time_elapsed = time.time() - since
            time_elapsed_phase = time.time() - phase_since
            print("Time: {:.0f}m {:.0f}s; Epoch {} in {:.0f}m {:.0f}s".format(
                time_elapsed // 60, time_elapsed % 60, phase,
                time_elapsed_phase // 60, time_elapsed_phase % 60
            ))

            # deep copy the model
            if phase == 'val' and epoch_mse < best_mse:
                best_mse = epoch_mse
                best_model_wts = copy.deepcopy(model.state_dict())

            loss_stat[phase].append([epoch, epoch_loss])
            
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val MSE: {:4f}'.format(best_mse))

    # load best model weights
    model_best = copy.deepcopy(model)
    model_best.load_state_dict(best_model_wts)
    return model_best, model

In [ ]:
model_ft = models.resnet18(pretrained=True)

# freeze all layers
for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
learning_rate = 1e-2
# optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=learning_rate, alpha=0.95, momentum=0.9, eps=0.0001)
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

In [ ]:
loss_stat = {
    'train': [],
    'train_': [],
    'val': []
}

model_best, model_last = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=21)

for phase in ['train', 'val', 'train_']:
    loss_stat[phase] = np.array(loss_stat[phase])

In [ ]:
plt.figure(figsize=(20,15))
for phase in ['train', 'val']:
    plt.plot(loss_stat[phase][:,0], loss_stat[phase][:,1], '-', label=phase, linewidth=3)
# plt.plot(loss_stat['train_'][::10,1], '-', label=phase, linewidth=1)
plt.xlabel('Epoch', fontsize=40)
plt.ylabel('Loss', fontsize=40)
plt.title('Loss during training', fontsize=50)
plt.tick_params(labelsize=30)
# plt.ylim(0,0.1)
plt.legend()
plt.grid()
plt.show()

In [ ]:
def data4visualization(loader, model, fps=30.0):
    y_true = []
    y_pred = []
    
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        k = 0
        it = iter(loader)
        sample = next(it, None)
        while sample != None:
            x, y = sample['frames'], sample['force']
            sample = next(it, None)
            
            # move to device, e.g. GPU
            x = x.to(device=device, dtype=dtype)  
            
            # predict
            у_pred_gpu = model(x)
            
            # store delta
            l = y.shape[0]
            y_pred_cpu = у_pred_gpu.data.cpu()
            y_pred.append(y_pred_cpu.numpy())
            y_true.append(y.numpy())
            k += l

    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
            
    y_pred = (y_pred + 0.5) * 30.0
    y_true = (y_true + 0.5) * 30.0
    
    t = np.arange(y_true.shape[0], dtype=np.float) / fps
    
    return t, y_true, y_pred

In [ ]:
t, y_true, y_pred = data4visualization(dataloaders['test'] , model_ft)

In [ ]:
linewidth = 5
def plot(t, y_true, y_pred, thumb=True, both=False, linewidth=2):
    plt.figure(figsize=(20,15))
    if thumb or both:
        plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
        plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')
    if not thumb or both:
        plt.plot(t, y_true[:,1], 'g.-', linewidth=linewidth, label='Index, ground truth')
        plt.plot(t, y_pred[:,1], 'k.-', linewidth=linewidth, label='Index, prediction')
    plt.ylabel('Force [N]', fontsize=32)
    plt.xlabel('Time [sec]', fontsize=32)
    plt.grid()
    plt.legend(fontsize=30)
    plt.tick_params(labelsize=24)
    plt.show()

In [ ]:
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
# model_ft = model_last

In [ ]:
# save the model
path = "/media/viktor/Samsung_T5/Research/models/05/transfer_6feb.pth.tar"
path_dict = "/media/viktor/Samsung_T5/Research/models/05/transfer_6feb_dict.pth.tar"
torch.save(model_ft, path)
torch.save(model_ft.state_dict, path_dict)

In [ ]:
# unfreeze all layers
for param in model_ft.parameters():
    param.requires_grad = True
    
loss_stat_freez = loss_stat.copy()    

loss_stat = {
    'train': [],
    'train_': [],
    'val': []
}

# Observe that all parameters are being optimized
learning_rate = 1e-3
# optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=learning_rate, alpha=0.95, momentum=0.9, eps=0.0001)
optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_best, model_last = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

for phase in ['train', 'val', 'train_']:
    loss_stat[phase] = np.array(loss_stat[phase])

## Load Model

In [ ]:
path = "/media/viktor/Samsung_T5/Research/models/01/transfer_25dec.pth.tar"
model_loaded = models.resnet18()
model_loaded = torch.load(path)
model_loaded.eval()

In [ ]:
t, y_true, y_pred = data4visualization(dataloaders['test'] , model_best)

In [ ]:
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
t, y_true, y_pred = data4visualization(dataloaders['test'] , model_last)

In [ ]:
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
threshold = 6.0

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

finger = 0

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)


mask = y_true[:,finger] <= threshold

print('%.2f' % r2_score(y_true[:,finger][mask], y_pred[:,finger][mask]))
rmse_astr = mean_squared_error(y_true[:,finger][mask], y_pred[:,finger][mask]) ** .5
print('%.2f' % rmse_astr)
print('%.1f' % (100 * rmse_astr / threshold))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

finger = 1

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)


mask = y_true[:,finger] <= threshold

print('%.2f' % r2_score(y_true[:,finger][mask], y_pred[:,finger][mask]))
rmse_astr = mean_squared_error(y_true[:,finger][mask], y_pred[:,finger][mask]) ** .5
print('%.2f' % rmse_astr)
print('%.1f' % (100 * rmse_astr / threshold))

In [ ]:
# Code fragments from https://www.bastibl.net/publication-quality-plots/

import numpy as np
import matplotlib as mpl
#mpl.use('pdf')
import matplotlib.pyplot as plt

plt.rc('font', family='serif', serif='Times')
# plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=14)

# width as measured in inkscape
width = 3.487
height = width / 1.618


fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

x = np.arange(0.0, 3*np.pi , 0.1)
plt.plot(x, np.sin(x))

ax.set_ylabel('Some Metric (in unit)')
ax.set_xlabel('Something (in unit)')
ax.set_xlim(0, 3*np.pi)

fig.show()
# fig.set_size_inches(width, height)
# fig.savefig('plot.pdf')

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

plt.plot(t, y_true[:,0], 'k', label='Ground truth')
plt.plot(t, y_pred[:,0], 'r', label='Estimation')


ax.set_ylabel('Force [N]')
ax.set_xlabel('Time [sec]')
# ax.set_xlim(5, 25)

plt.grid()
plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

plt.plot(t, y_true[:,1], 'k', label='Ground truth')
plt.plot(t, y_pred[:,1], 'b', label='Estimation')


ax.set_ylabel('Force [N]')
ax.set_xlabel('Time [sec]')
# ax.set_xlim(5, 25)

plt.grid()
plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

plt.plot(y_true[:,0], y_pred[:,0], 'r.', label='__')
# plt.plot(y_true[:,1], y_pred[:,1], 'b.', label='__')
plt.plot([0,43], [0,43], 'k-', label='__')


ax.set_xlabel('Force Ground Truth [N]')
ax.set_ylabel('Force Estimation [N]')

ax.set_xlim(0, 18)
ax.set_ylim(0, 18)

plt.grid()
# plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

# plt.plot(y_true[:,0], y_pred[:,0], 'r.', label='__')
plt.plot(y_true[:,1], y_pred[:,1], 'b.', label='__')
plt.plot([0,43], [0,43], 'k-', label='__')


ax.set_xlabel('Force Ground Truth [N]')
ax.set_ylabel('Force Estimation [N]')

ax.set_xlim(0, 18)
ax.set_ylim(0, 18)

plt.grid()
# plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

plt.plot(y_true[:,0], y_pred[:,0] - y_true[:,0], 'r.', label='__')
# plt.plot(y_true[:,1], y_pred[:,1] - y_true[:,0], 'b.', label='__')
plt.plot([0,43], [0,0], 'k-', label='__')


ax.set_xlabel('Force Ground Truth [N]')
ax.set_ylabel('Force Estimation Error [N]')

ax.set_xlim(0, 18)
ax.set_ylim(5, -6)

plt.grid()
# plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.15, bottom=.16, right=.99, top=.97)

# plt.plot(y_true[:,0], y_pred[:,0] - y_true[:,0], 'r.', label='__')
plt.plot(y_true[:,1], y_pred[:,1] - y_true[:,0], 'b.', label='__')
plt.plot([0,43], [0,0], 'k-', label='__')


ax.set_xlabel('Force Ground Truth [N]')
ax.set_ylabel('Force Estimation Error [N]')

ax.set_xlim(0, 18)
ax.set_ylim(6, -6)

plt.grid()
# plt.legend(fontsize=12)

# fig.set_size_inches(width, height)
fig.savefig('plot.png')

fig.show()

# plt.figure(figsize=(20,15))
# plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
# plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')

# plt.ylabel('Force [N]', fontsize=32)
# plt.xlabel('Time [sec]', fontsize=32)
# plt.grid()
# plt.legend(fontsize=30)
# plt.tick_params(labelsize=24)
# plt.show()